In [ ]:
# app.py
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.seasonal import STL
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# -----------------------------------------------------
# 🌟 Streamlit App Config
# -----------------------------------------------------
st.set_page_config(page_title="Stock Forecast | LSTM + STL", layout="wide", page_icon="📊")

st.title("📈 Stock Price Forecast using LSTM + STL Decomposition")
st.markdown("""
This interactive app uses **STL decomposition** and **LSTM neural networks**
to analyze and forecast stock prices.
You can adjust hyperparameters and see how it affects model performance.
""")

# -----------------------------------------------------
# 🔧 Sidebar Controls
# -----------------------------------------------------
st.sidebar.header("⚙️ Model Settings")

ticker = st.sidebar.text_input("Enter Stock Symbol", "HDFCBANK.NS")
start_date = st.sidebar.date_input("Start Date", datetime(2020, 1, 1))
end_date_forecast = st.sidebar.date_input("End Date", datetime(2025, 8, 6))
neurons = st.sidebar.slider("Number of LSTM Neurons", min_value=5, max_value=100, value=10, step=5)
epochs = st.sidebar.slider("Training Epochs", min_value=5, max_value=50, value=10, step=5)
window_size = st.sidebar.slider("Window Size", min_value=5, max_value=30, value=10, step=1)

st.sidebar.markdown("---")
st.sidebar.info("Click below to run the model 👇")
run_button = st.sidebar.button("🚀 Run Forecast")

# -----------------------------------------------------
# 🧠 Function Definitions
# -----------------------------------------------------
def prepare_lstm_data(series, window_size):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(series.values.reshape(-1, 1))
    X, y = [], []
    for i in range(len(scaled) - window_size):
        X.append(scaled[i:i+window_size])
        y.append(scaled[i+window_size])
    return np.array(X), np.array(y), scaler


def build_and_train_lstm(X, y, neurons=10, epochs=10):
    model = Sequential([
        LSTM(neurons, return_sequences=True, input_shape=(X.shape[1], 1)),
        LSTM(neurons, return_sequences=True),
        LSTM(neurons, return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=epochs, batch_size=10, verbose=0)
    return model


# -----------------------------------------------------
# 🚀 Run Forecast
# -----------------------------------------------------
if run_button:
    st.info("Downloading stock data... ⏳")

    data = yf.download(ticker, start=start_date, end=end_date_forecast)

    if data.empty:
        st.error("No data found for this symbol and date range.")
    else:
        d_high = data["High"]

        st.subheader(f"📊 Data Overview for {ticker}")
        st.write(f"**Records:** {len(d_high)} | **From:** {start_date} → **To:** {end_date_forecast}")
        st.dataframe(d_high.tail())

        # -----------------------------------------------------
        # 🔹 STL Decomposition
        # -----------------------------------------------------
        st.subheader("🧩 STL Decomposition")
        with st.spinner("Decomposing time series..."):
            stl = STL(d_high, period=30)
            result = stl.fit()
            trend, seasonal, resid = result.trend, result.seasonal, result.resid

        fig, axs = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
        axs[0].plot(d_high, color='blue', label='Original')
        axs[1].plot(trend, color='green', label='Trend')
        axs[2].plot(seasonal, color='orange', label='Seasonal')
        axs[3].plot(resid, color='red', label='Residual')
        for ax in axs:
            ax.legend()
            ax.grid(True)
        st.pyplot(fig)

        # -----------------------------------------------------
        # 🧮 Prepare Data for LSTM
        # -----------------------------------------------------
        st.subheader("📦 Preparing Data for LSTM")
        X_trend, y_trend, scaler_trend = prepare_lstm_data(trend, window_size)
        X_seasonal, y_seasonal, scaler_seasonal = prepare_lstm_data(seasonal, window_size)

        # -----------------------------------------------------
        # 🧠 Train LSTM Models
        # -----------------------------------------------------
        st.subheader("🤖 Training LSTM Models")
        with st.spinner("Training trend and seasonal models..."):
            model_trend = build_and_train_lstm(X_trend, y_trend, neurons=neurons, epochs=epochs)
            model_seasonal = build_and_train_lstm(X_seasonal, y_seasonal, neurons=neurons, epochs=epochs)

        # -----------------------------------------------------
        # 📊 Predictions
        # -----------------------------------------------------
        y_trend_pred = model_trend.predict(X_trend)
        y_seasonal_pred = model_seasonal.predict(X_seasonal)

        trend_pred = scaler_trend.inverse_transform(y_trend_pred)
        seasonal_pred = scaler_seasonal.inverse_transform(y_seasonal_pred)
        final_pred = trend_pred.flatten() + seasonal_pred.flatten()

        actual = d_high.values[window_size:]

        # -----------------------------------------------------
        # 📈 Visualization
        # -----------------------------------------------------
        st.subheader("📈 Actual vs Predicted")
        fig2, ax = plt.subplots(figsize=(10, 5))
        ax.plot(actual, label="Actual", color="blue")
        ax.plot(final_pred, label="Predicted (Trend + Seasonal)", color="red")
        ax.set_title(f"LSTM Forecast (Neurons={neurons}, Epochs={epochs})")
        ax.legend()
        ax.grid(True)
        st.pyplot(fig2)

        # -----------------------------------------------------
        # 📉 Evaluation
        # -----------------------------------------------------
        rmse = np.sqrt(mean_squared_error(actual, final_pred))
        st.metric(label="📉 RMSE (Model Error)", value=f"{rmse:.4f}")

        # -----------------------------------------------------
        # 🔮 1-Step Forecast
        # -----------------------------------------------------
        st.subheader("🔮 Next Day Forecast")

        last_trend_window = trend.values[-window_size:].reshape(1, window_size, 1)
        last_trend_scaled = scaler_trend.transform(last_trend_window.reshape(window_size, 1)).reshape(1, window_size, 1)
        next_trend_scaled = model_trend.predict(last_trend_scaled)
        next_trend = scaler_trend.inverse_transform(next_trend_scaled)[0][0]

        last_seasonal_window = seasonal.values[-window_size:].reshape(1, window_size, 1)
        last_seasonal_scaled = scaler_seasonal.transform(last_seasonal_window.reshape(window_size, 1)).reshape(1, window_size, 1)
        next_seasonal_scaled = model_seasonal.predict(last_seasonal_scaled)
        next_seasonal = scaler_seasonal.inverse_transform(next_seasonal_scaled)[0][0]

        next_day_forecast = next_trend + next_seasonal
        st.success(f"📅 Forecasted Next Day High: **{next_day_forecast:.4f}**")

        st.markdown("---")
        st.caption("Developed with ❤️ using Streamlit, TensorFlow & Statsmodels.")
